In [1]:
%%capture
%pip install transformers SentencePiece accelerate huggingface_hub

In [2]:
import transformers, torch
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/Adrastea-7b-v1.0")
model = LlamaForCausalLM.from_pretrained(
        "decapoda-research/Adrastea-7b-v1.0",
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [4]:
eos_token_id = tokenizer.eos_token_id
pad_token_id = tokenizer.pad_token_id


instruction = "Give me a Jenkins pipeline script"
inputs = tokenizer(
    f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: {instruction}
### Response:""",
    return_tensors="pt",
)
input_ids = inputs["input_ids"].to("cuda")

generation_config = transformers.GenerationConfig(
    do_sample=True,
    temperature=0.1,
    top_p=0.75,
    top_k=80,
    repetition_penalty=1.5,
    max_new_tokens=128,
)

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
        eos_token_id=eos_token_id,
        pad_token_id=pad_token_id,
    )
output_text = tokenizer.decode(
    generation_output[0].cuda(), skip_special_tokens=True
).strip()
print(output_text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: Give me a Jenkins pipeline script
### Response: 1) Create your own job in Jenkins and select Pipeline as the type of project you want to create, then click OK.  2) In the Source Code Management section under Git (or any other SCM), provide repository URL or path where your code resides on Github/Bitbucket etc., along with credentials if required for accessing it. Then Click Save button at bottom right corner.
